# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import RandomizedSearchCV 

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [ ]:
sales_smooth

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
1615,FOODS_1_004_CA_1_validation,FOODS_1_004,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1622,FOODS_1_012_CA_1_validation,FOODS_1_012,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,1,0,6,5,2,2,5,6,2,4
1628,FOODS_1_018_CA_1_validation,FOODS_1_018,FOODS_1,FOODS,CA_1,CA,17,3,8,2,...,4,7,6,6,5,1,9,25,22,13
1653,FOODS_1_043_CA_1_validation,FOODS_1_043,FOODS_1,FOODS,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1656,FOODS_1_046_CA_1_validation,FOODS_1_046,FOODS_1,FOODS,CA_1,CA,0,0,0,5,...,0,7,6,13,13,6,9,7,10,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12153,FOODS_3_785_CA_4_validation,FOODS_3_785,FOODS_3,FOODS,CA_4,CA,3,0,1,1,...,0,0,0,0,0,11,6,7,7,23
12163,FOODS_3_795_CA_4_validation,FOODS_3_795,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,1,2,3,1,3,3,2,7,3,3
12175,FOODS_3_807_CA_4_validation,FOODS_3_807,FOODS_3,FOODS,CA_4,CA,4,5,9,7,...,0,0,0,0,0,0,0,0,0,0
12179,FOODS_3_811_CA_4_validation,FOODS_3_811,FOODS_3,FOODS,CA_4,CA,0,0,0,0,...,2,7,4,1,0,0,2,3,5,2


# User-defined functions to calculate Metrics

In [8]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [ ]:
def data_cleaning(sales_pattern):

    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()

    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')

    df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
    df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
    df_pattern.reset_index(inplace=True)

    calendar['Date'] = pd.to_datetime(calendar['date'])

    dataframe = pd.merge(df_pattern,
                         calendar[['wday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'Date']],
                         on = 'Date')

    dataframe['weekend'] = np.where(((dataframe['wday'] == 1) | (dataframe['wday'] == 2)), 1, 0)
    dataframe.fillna('No event', inplace=True)
    dataframe['year'] = dataframe['year'].apply(str)

    dataframe['quarter'] = dataframe['Date'].dt.quarter
    dataframe['quarter'] = dataframe['quarter'].apply(str)

    dataframe = pd.get_dummies(dataframe)
    
    return dataframe

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [ ]:
data_pattern = data_cleaning(sales_smooth)

In [ ]:
data_pattern

,Date,FOODS_1_004_CA_1,FOODS_1_004_CA_3,FOODS_1_004_CA_4,FOODS_1_012_CA_1,FOODS_1_012_CA_2,FOODS_1_012_CA_3,FOODS_1_012_CA_4,FOODS_1_018_CA_1,FOODS_1_018_CA_4,...,event_name_1_VeteransDay,event_type_1_Cultural,event_type_1_National,event_type_1_No event,event_type_1_Religious,event_type_1_Sporting,quarter_1,quarter_2,quarter_3,quarter_4
0,2011-01-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,2.0,...,0,0,0,1,0,0,1,0,0,0
1,2011-01-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0,0,0,1,0,0,1,0,0,0
2,2011-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,...,0,0,0,1,0,0,1,0,0,0
3,2011-02-01,4.0,0.0,1.0,4.0,21.0,11.0,3.0,8.0,8.0,...,0,0,0,1,0,0,1,0,0,0
4,2011-02-02,9.0,0.0,3.0,7.0,24.0,18.0,8.0,14.0,6.0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,10.0,0.0,1.0,7.0,8.0,7.0,10.0,13.0,8.0,...,0,0,0,1,0,0,0,1,0,0
1909,2016-04-21,3.0,0.0,1.0,2.0,6.0,11.0,1.0,15.0,7.0,...,0,0,0,1,0,0,0,1,0,0
1910,2016-04-22,7.0,0.0,2.0,5.0,13.0,10.0,3.0,14.0,9.0,...,0,0,0,1,0,0,0,1,0,0
1911,2016-04-23,6.0,0.0,3.0,4.0,9.0,7.0,6.0,15.0,7.0,...,0,0,0,1,0,0,0,1,0,0


# Random Forest Hyperparameters to find optimum parameters

In [ ]:
not_id_cols = [
 'Date',
 'wday',
 'month',
 'snap_CA',
 'weekend',
 'year_2011',
 'year_2012',
 'year_2013',
 'year_2014',
 'year_2015',
 'year_2016',
 'event_name_1_Chanukah End',
 'event_name_1_Christmas',
 'event_name_1_Cinco De Mayo',
 'event_name_1_ColumbusDay',
 'event_name_1_Easter',
 'event_name_1_Eid al-Fitr',
 'event_name_1_EidAlAdha',
 "event_name_1_Father's day",
 'event_name_1_Halloween',
 'event_name_1_IndependenceDay',
 'event_name_1_LaborDay',
 'event_name_1_LentStart',
 'event_name_1_LentWeek2',
 'event_name_1_MartinLutherKingDay',
 'event_name_1_MemorialDay',
 "event_name_1_Mother's day",
 'event_name_1_NBAFinalsEnd',
 'event_name_1_NBAFinalsStart',
 'event_name_1_NewYear',
 'event_name_1_No event',
 'event_name_1_OrthodoxChristmas',
 'event_name_1_OrthodoxEaster',
 'event_name_1_Pesach End',
 'event_name_1_PresidentsDay',
 'event_name_1_Purim End',
 'event_name_1_Ramadan starts',
 'event_name_1_StPatricksDay',
 'event_name_1_SuperBowl',
 'event_name_1_Thanksgiving',
 'event_name_1_ValentinesDay',
 'event_name_1_VeteransDay',
 'event_type_1_Cultural',
 'event_type_1_National',
 'event_type_1_No event',
 'event_type_1_Religious',
 'event_type_1_Sporting',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4'
]

In [ ]:
end_train_date = '2016-04-10'

In [ ]:
lags = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]
moving_averages = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
stds = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
maximums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
minimums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
totals = [2, 3, 4, 5, 6, 7, 14, 21, 28]
num_days_zeros = [2, 3, 4, 5, 6, 7, 14, 21, 28] 

In [ ]:
correlation_threshold = 0.8

In [ ]:
n_estimators = [50, 100, 200, 300]
max_depth = list(range(4,15)) + [None]
min_samples_leaf = range(2,12)
max_features = range(10,30)
bootstrap = [True] #We force bootstrap
max_samples = [0.6, 0.7, 0.8, 0.9, 1]

In [ ]:
# User-defined funtion to find optimum parameters on training data
def Random_Forest_Pipeline(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date, n_estimators, max_depth, min_samples_leaf,
                          max_features, bootstrap, max_samples):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_hyper_params = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running Product: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))
        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        param_dist = {'n_estimators': n_estimators,
                      'max_depth': max_depth,
                      'min_samples_leaf': min_samples_leaf,
                      'max_features': max_features,
                      'bootstrap': bootstrap,
                      'max_samples': max_samples}

        forest = RandomForestRegressor(n_jobs=1)

        forest_cv = RandomizedSearchCV(forest, param_dist, cv=5, n_jobs=-1, verbose=2, n_iter=300, scoring='neg_mean_absolute_error')
        forest_cv.fit(X_train,y_train)
            
        best_params_temp = forest_cv.best_params_
        
        df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
        df_output_temp['Product'] = product
        
        df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)
        
    return df_output_hyper_params

**Sample 30 products due to limited computational power**

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [ ]:
list_pattern_sample = sample(list_smooth, 30)

In [ ]:
list_pattern_sample

['FOODS_3_228_CA_2_validation',
 'FOODS_3_227_CA_1_validation',
 'FOODS_3_291_CA_3_validation',
 'FOODS_3_330_CA_1_validation',
 'FOODS_3_459_CA_1_validation',
 'FOODS_1_023_CA_4_validation',
 'FOODS_3_725_CA_1_validation',
 'FOODS_2_183_CA_1_validation',
 'FOODS_2_054_CA_1_validation',
 'FOODS_3_473_CA_1_validation',
 'FOODS_1_023_CA_3_validation',
 'FOODS_2_286_CA_2_validation',
 'FOODS_3_058_CA_3_validation',
 'FOODS_3_259_CA_3_validation',
 'FOODS_3_090_CA_1_validation',
 'FOODS_3_232_CA_4_validation',
 'FOODS_3_217_CA_4_validation',
 'FOODS_3_369_CA_3_validation',
 'FOODS_3_565_CA_3_validation',
 'FOODS_3_344_CA_1_validation',
 'FOODS_3_665_CA_3_validation',
 'FOODS_3_625_CA_1_validation',
 'FOODS_3_300_CA_3_validation',
 'FOODS_3_121_CA_1_validation',
 'FOODS_3_546_CA_1_validation',
 'FOODS_3_277_CA_1_validation',
 'FOODS_3_708_CA_4_validation',
 'FOODS_1_218_CA_1_validation',
 'FOODS_3_403_CA_3_validation',
 'FOODS_2_299_CA_3_validation']

**Start Hyperparameters tuning process**

In [ ]:
start = timer()

df_hyper_params = Random_Forest_Pipeline(list_pattern_sample, not_id_cols, lags, moving_averages, stds,
                                         maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                         end_train_date, n_estimators, max_depth, min_samples_leaf,
                                         max_features, bootstrap, max_samples)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_3_131_CA_1
Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Currently Running Product: FOODS_3_258_CA_4
Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Currently Running Product: FOODS_3_277_CA_1
Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Currently Running Product: FOODS_3_113_CA_1
Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Currently Running Product: FOODS_3_201_CA_1
Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Currently Running Product: FOODS_3_121_CA_3
Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Currently Running Product: FOODS_2_299_CA_4
Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Currently Running Product: FOODS_2_304_CA_3
Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Currently Running Product: FOODS_3_226_CA_1
Fitting 5 folds for each of 400 candidates, totalling 2000 fits
Currently Running Product: F

In [ ]:
df_hyper_params

,n_estimators,min_samples_leaf,max_samples,max_features,max_depth,bootstrap,Product
0,50,5,0.8,25,13,True,FOODS_3_131_CA_1
1,200,3,0.6,19,13,True,FOODS_3_258_CA_4
2,50,3,0.6,28,4,True,FOODS_3_277_CA_1
3,50,6,0.7,24,7,True,FOODS_3_113_CA_1
4,50,8,0.8,13,10,True,FOODS_3_201_CA_1
5,50,5,0.6,25,6,True,FOODS_3_121_CA_3
6,50,8,1,10,9,True,FOODS_2_299_CA_4
7,300,6,0.8,26,5,True,FOODS_2_304_CA_3
8,50,11,0.9,12,9,True,FOODS_3_226_CA_1
9,100,10,0.9,15,None,True,FOODS_2_204_CA_3


In [ ]:
df_hyper_params.to_csv('Random_Forest_Hyperparams_Smooth.csv')

# Decide optimum set of parameters for Random Forest algorithm

In [ ]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_hyper_params.groupby(['n_estimators', 'min_samples_leaf', 'max_samples', 
                         'max_features', 'max_depth', 'bootstrap'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
n_estimators min_samples_leaf max_samples max_features max_depth bootstrap         
50           3                0.6         28           4.0       True             1
             4                0.6         22           10.0      True             1
300          5                0.7         24           4.0       True             1
             3                0.6         25           6.0       True             1
200          11               0.9         29           4.0       True             1
             8                0.9         29           12.0      True             1
             3                0.6         19           13.0      True             1
             2                0.7         25           4.0       True             1
100          11               0.8         29           12.0      True             1
             10               0.6         21           12.0      True             1
             9                0.9         21           6.0       True             1
             8                0.6         26           10.0      True             1
             6                0.7         23           4.0       True             1
50           11               0.9         12           9.0       True             1
             10               0.6         24           9.0       True             1
             9                0.7         14           9.0       True             1
                              0.6         21           7.0       True             1
             8                1.0         10           9.0       True             1
                              0.8         13           10.0      True             1
             7                1.0         13           13.0      True             1
             6                0.7         24           7.0       True             1
             5                0.9         24           6.0       True             1
                              0.8         25           13.0      True             1
                              0.6         25           6.0       True             1
             4                0.7         19           5.0       True             1
300          6                0.8         26           5.0       True             1

In [ ]:
# See what value of parameter "n_estimators" occurs the most frequently
df_hyper_params.groupby(['n_estimators'])[['Product']].count()

,Product
n_estimators,
50,15
100,6
200,4
300,5


In [ ]:
# See what value of parameter "min_samples_leaf" occurs the most frequently
df_hyper_params.groupby(['min_samples_leaf'])[['Product']].count()

,Product
min_samples_leaf,
2,1
3,3
4,2
5,5
6,4
7,1
8,4
9,4
10,3


In [ ]:
# See what value of parameter "max_samples" occurs the most frequently
df_hyper_params.groupby(['max_samples'])[['Product']].count()

,Product
max_samples,
0.6,10
0.7,7
0.8,5
0.9,6
1.0,2


In [ ]:
# See what value of parameter "max_features" occurs the most frequently
df_hyper_params.groupby(['max_features'])[['Product']].count()

,Product
max_features,
10,1
12,1
13,2
14,1
15,1
19,2
21,4
22,1
23,1


In [ ]:
# See what value of parameter "max_depth" occurs the most frequently
df_hyper_params.groupby(['max_depth'])[['Product']].count()

,Product
max_depth,
4.0,5
5.0,2
6.0,4
7.0,2
9.0,4
10.0,3
12.0,3
13.0,3


# Implement Random Forest on Test Set with optimum set of parameters

**Specify optimum set of parameters**

In [ ]:
n_estimators_chosen = 50
min_samples_leaf_chosen = 5
max_samples_choose = 0.6
max_features_choose = 21
max_depth_choose = 6
bootstrap = [True]

In [ ]:
# User-defined function to run Random Forest on Test Data

def Random_Forest_Test_Data(list_pattern, not_id_cols, lags, moving_averages, stds,
                          maximums, minimums, totals, num_days_zeros, correlation_threshold,
                          end_train_date, n_estimators_chosen, min_samples_leaf_chosen,
                          max_samples_choose, max_features_choose, max_depth_choose, bootstrap):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_rf_pattern = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running Product: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]
        test_data = dataframe_product[dataframe_product.index > end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        X_test = test_data.drop(product, axis=1)
        y_test = test_data[product]

        forest = RandomForestRegressor(n_jobs=1,
                                      n_estimators = n_estimators_chosen,
                                      min_samples_leaf = min_samples_leaf_chosen,
                                      max_samples = max_samples_choose,
                                      max_features = max_features_choose,
                                      max_depth = max_depth_choose,
                                      bootstrap = bootstrap,
                                      random_state = 0)

        forest.fit(X_train,y_train)
        y_pred = forest.predict(X_test)
        df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                       'Forecast': y_pred, 
                                       'Product': [product for num_rows in range(len(y_pred))]
                                       })
        
        df_output_rf_pattern = df_output_rf_pattern.append(df_output_temp, ignore_index=False)
        
    return df_output_rf_pattern

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [ ]:
# Start running Random Forest on Test Set

start = timer()

df_forecast_values = Random_Forest_Test_Data(list_smooth, not_id_cols, lags, moving_averages, stds,
                                             maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                             end_train_date, n_estimators_chosen, min_samples_leaf_chosen,
                                             max_samples_choose, max_features_choose, max_depth_choose, bootstrap)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: FOODS_1_004_CA_1
Progressing: 0.0 %
Currently Running Product: FOODS_1_004_CA_3
Progressing: 0.16 %
Currently Running Product: FOODS_1_004_CA_4
Progressing: 0.31 %
Currently Running Product: FOODS_1_012_CA_1
Progressing: 0.47 %
Currently Running Product: FOODS_1_012_CA_2
Progressing: 0.62 %
Currently Running Product: FOODS_1_012_CA_3
Progressing: 0.78 %
Currently Running Product: FOODS_1_012_CA_4
Progressing: 0.93 %
Currently Running Product: FOODS_1_018_CA_1
Progressing: 1.09 %
Currently Running Product: FOODS_1_018_CA_4
Progressing: 1.25 %
Currently Running Product: FOODS_1_019_CA_4
Progressing: 1.4 %
Currently Running Product: FOODS_1_023_CA_3
Progressing: 1.56 %
Currently Running Product: FOODS_1_023_CA_4
Progressing: 1.71 %
Currently Running Product: FOODS_1_024_CA_2
Progressing: 1.87 %
Currently Running Product: FOODS_1_043_CA_1
Progressing: 2.02 %
Currently Running Product: FOODS_1_043_CA_2
Progressing: 2.18 %
Currently Running Product: FOODS_1_043_CA_

In [ ]:
df_forecast_values

,Actual Data,Forecast,Product
Date,,,
2016-04-11,2.0,5.746389,FOODS_1_004_CA_1
2016-04-12,11.0,4.942159,FOODS_1_004_CA_1
2016-04-13,8.0,7.307019,FOODS_1_004_CA_1
2016-04-14,0.0,7.130154,FOODS_1_004_CA_1
2016-04-15,10.0,6.500998,FOODS_1_004_CA_1
...,...,...,...
2016-04-20,0.0,0.571071,FOODS_3_823_CA_3
2016-04-21,0.0,0.591803,FOODS_3_823_CA_3
2016-04-22,0.0,0.753451,FOODS_3_823_CA_3


In [ ]:
df_forecast_values.to_csv('Random Forest_Smooth_Forecast_Values.csv')

In [5]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [9]:
start = timer()

df_forecast_metrics = df_forecast_values.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.023951742033333782 minutes


In [10]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_1,0.782210,68.6723,71.3149,108.2471,4.512753,5.442973
1,FOODS_1_004_CA_3,inf,inf,200.0000,NaN,0.415747,0.462789
2,FOODS_1_004_CA_4,0.822815,49.5178,56.9840,57.9465,1.202576,1.515500
3,FOODS_1_012_CA_1,0.710846,49.5341,61.0173,62.5711,1.804455,2.032024
4,FOODS_1_012_CA_2,0.674737,39.6783,43.8107,35.4504,4.307933,5.195139
...,...,...,...,...,...,...,...
637,FOODS_3_818_CA_4,0.592805,44.0279,47.9722,46.0045,0.912007,1.147237
638,FOODS_3_819_CA_3,0.608952,48.8631,50.9132,98.3790,1.780015,2.106628
639,FOODS_3_820_CA_1,0.569104,53.8593,67.8684,44.1737,1.269540,1.512031
640,FOODS_3_820_CA_3,0.674791,69.6419,84.6878,41.6861,1.193860,1.413668


In [11]:
df_forecast_metrics.to_csv('Random Forest_Smooth_Forecast_Metrics.csv')